In [2]:
library(gridExtra)
library(matrixStats)
library(dplyr)
library(rlang)
library(reshape2)
library(clusterCrit)
library(grid)
library(ggrepel)
library(RColorBrewer)
library(dplyr)
library(IRdisplay)
library(scales)
library(survival)
library(corrplot)
library(Hmisc)
library(ggplot2)
library(dummies)
source('../../../src/tools.R')     # custom tools function
source('../../../src/hdp_tools_yanis.R')
source("../../../src/merge_df.R")
source("../../../src/my_toolbox.R")
source("../../../src/my_components.R")
source("../../../src/my_utils.R")
source("../../../src/ggstyles.R")
source("../../../src/my_hotspots.R")
###


theme_set(theme_minimal())

# set jupyer notebook parameters
options(repr.plot.res        = 100, # set a medium-definition resolution for the jupyter notebooks plots (DPI)
        repr.matrix.max.rows = 200, # set the maximum number of rows displayed
        repr.matrix.max.cols = 200) # set the maximum number of columns displayed


Attaching package: ‘dplyr’

The following object is masked from ‘package:matrixStats’:

    count

The following object is masked from ‘package:gridExtra’:

    combine

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: ggplot2

Attaching package: ‘survival’

The following object is masked from ‘package:clusterCrit’:

    concordance

corrplot 0.84 loaded
Loading required package: lattice
Loading required package: Formula

Attaching package: ‘Hmisc’

The following objects are masked from ‘package:dplyr’:

    src, summarize

The following objects are masked from ‘package:base’:

    format.pval, units

dummies-1.5.6 provided by Decision Patterns

Run citation('hdp') for citation instructions,
    and file.show(system.file('LICENSE', package='hdp')) for license details.

Attaching package: ‘DescTools’

The following objects are masked from ‘pa

In [17]:
df_final <- read.table("df_final_with_comp.tsv")[,c(1:156)]

In [5]:
list_component <- unique(df_final[order(df_final$final_component_numeric),]$final_component)
list_component

[1] t_15_17               inv_16                t_8_21               
 [4] t_11                  t_6_9                 inv_3                
 [7] additions             TP53_complex          NPM1                 
[10] CEBPA_bi              DNMT3A_IDH1_2         WT1                  
[13] chr_splicing_multiple chr_splicing_1        not_assigned         
[16] no_events            
16 Levels: additions CEBPA_bi chr_splicing_1 ... WT1

In [54]:
tmp <- df_final
factors <- c("ZRSR2","U2AF1_p.S34","U2AF1_p.Q157","SRSF2","SF3B1","ASXL1","STAG2","BCOR","RUNX1","EZH2","MLL","PHF6","SF1","NF1","CUX1","SETBP1")
tmp$comp_t_15_17 <- ifelse((df_final$t_15_17==1),1,0) 
tmp$comp_inv_16 <- ifelse((df_final$inv_16==1),1,0) 
tmp$comp_t_8_21 <- ifelse((df_final$t_8_21==1),1,0) 
tmp$comp_t_11 <- ifelse((df_final$t_v_11==1 | df_final$t_9_11==1),1,0)   
tmp$comp_t_6_9 <- ifelse((df_final$t_6_9==1),1,0) 
tmp$comp_inv_3 <- ifelse((df_final$inv_3==1),1,0)
tmp$comp_additions <- ifelse((df_final$add_8==1 | df_final$add_13==1 | df_final$add_21==1 | df_final$add_22==1),1,0)
tmp$comp_TP53_complex <- ifelse((df_final$TP53==1 | df_final$complex==1) ,1,0)
tmp$comp_NPM1 <- ifelse((df_final$NPM1==1),1,0) 
tmp$comp_CEBPA_bi <- ifelse((df_final$CEBPA_bi==1),1,0) 
tmp$comp_DNMT3A_IDH1_2 <- ifelse((df_final$DNMT3A==1 & (df_final$IDH1==1 | df_final$IDH2_p.R140==1 | df_final$IDH2_p.R172==1)),1,0)
tmp$comp_WT1 <- ifelse((df_final$WT1==1),1,0) 
tmp$comp_chr_splicing_multiple <- ifelse((rowSums(df_final[,factors])>1) ,1,0)
tmp$comp_chr_splicing_1 <- ifelse((rowSums(df_final[,factors])==1),1,0)
tmp$comp_not_assigned <- ifelse(df_final$final_component=="not_assigned",1,0)
tmp$comp_no_events <- ifelse(df_final$final_component=="no_events",1,0)

tmp$num_comp_fit <- rowSums(tmp[,c(157:172)])
tmp$overlap <- ifelse(tmp$num_comp_fit>1,1,0)
    
#write.table(tmp,"df_final_with_multiple_comp.tsv")   

In [27]:
correlates <- c("ahd","perf_status","bm_blasts","secondary","wbc","hb","plt","gender","age","os","os_status")
df_final <- read.table("df_final_with_multiple_comp.tsv")
for (col in colnames(df_final[,startsWith(colnames(df_final),"comp_")])){
    df_final[,gsub("comp_","overlap_",col)] <- 0 
}
### Components t_15_17,inv_16,t_8_21,t_11,t_6_9,inv_3,NPM1,

for (col in c("t_15_17","inv_16","t_8_21","t_11","t_6_9","inv_3","NPM1","CEBPA_bi","TP53_complex")){
    for (col_bis in colnames(df_final[,startsWith(colnames(df_final),"comp_")])){
        if (!is.element(col_bis,c("comp_additions","comp_WT1","comp_chr_splicing_1","comp_DNMT3A_IDH1_2",paste("comp",col,sep="_")))){
            try(df_final[df_final$final_component==col  , gsub("comp_","overlap_",col_bis)] <- df_final[df_final$final_component==col,col_bis])
        }
    }      
}

### Component additions
df_final[df_final$final_component=="additions" & df_final$comp_TP53_complex==1 & (df_final$del_7==1 | df_final$del_17==1 | df_final$del_5==1 | df_final$del_9==1 |
         df_final$del_13==1 | df_final$del_20==1 | df_final$del_18==1 | df_final$del_16==1 | df_final$del_12==1 | df_final$del_3==1),"overlap_TP53_complex"] <- 1
df_final[df_final$final_component=="additions" & df_final$comp_chr_splicing_multiple==1,"overlap_chr_splicing_multiple"] <- 1
df_final[df_final$final_component=="additions" & df_final$comp_chr_splicing_1==1,"overlap_chr_splicing_1"] <- 1

### Component TP53 complex continuation
df_final[df_final$final_component=="TP53_complex" & df_final$comp_additions==1 & (df_final$del_7==1 | df_final$del_17==1 | df_final$del_5==1 | df_final$del_9==1 |
         df_final$del_13==1 | df_final$del_20==1 | df_final$del_18==1 | df_final$del_16==1 | df_final$del_12==1 | df_final$del_3==1),"overlap_additions"] <- 1

### Component DNMT3A_IDH1_2
df_final[df_final$final_component=="DNMT3A_IDH1_2" & df_final$comp_WT1==1,"overlap_WT1"] <- 1
df_final[df_final$final_component=="DNMT3A_IDH1_2" & df_final$comp_chr_splicing_multiple==1,"overlap_chr_splicing_multiple"] <- 1
df_final[df_final$final_component=="DNMT3A_IDH1_2" & df_final$comp_chr_splicing_1==1,"overlap_chr_splicing_1"] <- 1

### Component WT1
df_final[df_final$final_component=="WT1" & df_final$comp_DNMT3A_IDH1_2==1,"overlap_DNMT3A_IDH1_2"] <- 1
df_final[df_final$final_component=="WT1" & df_final$comp_chr_splicing_multiple==1,"overlap_chr_splicing_multiple"] <- 1
df_final[df_final$final_component=="WT1" & df_final$comp_chr_splicing_1==1,"overlap_chr_splicing_1"] <- 1

### Component chr splicing 1
df_final[df_final$final_component=="chr_splicing_1" & df_final$comp_additions==1,"overlap_additions"] <- 1
df_final[df_final$final_component=="chr_splicing_1" & df_final$comp_WT1==1,"overlap_WT1"] <- 1
df_final[df_final$final_component=="chr_splicing_1" & df_final$comp_DNMT3A_IDH1_2==1,"overlap_DNMT3A_IDH1_2"] <- 1

### Component chr splicing multiple
for (col in colnames(df_final[,startsWith(colnames(df_final),"comp_")])){
    if(col!="comp_chr_splicing_multiple"){
        df_final[df_final$final_component=="chr_splicing_multiple"  , gsub("comp_","overlap_",col)] <- df_final[df_final$final_component=="chr_splicing_multiple",col]
    }
}
#write.table(df_final,"df_final_multiple_comp_overlap_comp.tsv")

NULL

In [39]:
correlates <- c("ahd","perf_status","bm_blasts","secondary","wbc","hb","plt","gender","age","os","os_status")
continuous_correlates <- c("age","wbc","hb","plt","bm_blasts","os")
df_all_components <- read.table("../../clustering/Clustering_Final_1/df_final_multiple_comp_overlap_comp.tsv")[,c(1:156,175:190)]

df_initial <- read.table("../../../data/initial_dataset/Master_04_10_2019.csv",sep = ',' , header = T)
rownames(df_initial) <- df_initial$data_pd
df_initial <- df_initial[,-1:-4]
df_eln <- read.table("../../../data/updated_dataset/eln_final.tsv",sep = '\t' , header = T)
rownames(df_eln) <- df_eln$X
df_eln <- df_eln[-1]
df <- merge(df_eln,df_all_components,by=0)
rownames(df) <- df$Row.names
df <- df[-1]
df <- merge(df,df_initial[,correlates],by=0)
rownames(df) <- df$Row.names
df <- df[-1]
for (col in colnames(df)[colSums(is.na(df)) > 0]){
    if(!is.element(col,c("os","os_status"))){        
        if(is.element(col,continuous_correlates)){
            df[,col][is.na(df[,col])] <- mean(df[,col], na.rm=TRUE)
        }else{
            df[,col][is.na(df[,col])] <- median(df[,col], na.rm=TRUE)
            }
    }
   
}
df <- na.omit(df)
df <- df[df$os>0,]

df$eln_2017 <- ifelse(df$eln_2017=="adverse",1,
                    ifelse(df$eln_2017=="intermediate",2,3))
df<- cbind(df,dummy('final_component',df, sep = "_"))
overlap_comp <- colnames(df[,startsWith(colnames(df),"overlap_"),])
df$overlap <- 0
for (comp in colnames(df[,startsWith(colnames(df),"final_component_"),])){
    df[df[,comp]==1,]$overlap <- ifelse(rowSums(df[df[,comp]==1,overlap_comp])>0,1,0)
}
df <- df[,c(1:158,177:185,159,160,188:204,161:176,186:187)]
df$overlap_NPM1 <- NULL
df$overlap_not_assigned <- NULL
df$overlap_no_events <- NULL
for (col in (colnames(df[,startsWith(colnames(df),"final_component_"),]))){
    if(col!="final_component_numeric"){
        if (is.element(col,c('final_component_not_assigned','final_component_no_events','final_component_NPM1'))){
            df[,gsub('final','full',col)] <- df[,col]
        }
        else{
            df[,gsub('final','full',col)] <- df[,col] + df[,gsub('final_component','overlap',col)]
        }
    }    
}
df <- df[,c(1:199,202:217,200:201)]
write.table(df,"df_final_analysis.tsv")

In [148]:
# Final Table to use



df_final <- read.table("df_final_with_comp.tsv")[,c(1:156)]
tmp <- df_final
factors <- c("ZRSR2","U2AF1_p.S34","U2AF1_p.Q157","SRSF2","SF3B1","ASXL1","STAG2","BCOR","RUNX1","EZH2","MLL","PHF6","SF1","NF1","CUX1","SETBP1")
correlates <- c("ahd","perf_status","bm_blasts","secondary","wbc","hb","plt","gender","age","os","os_status")
tmp$full_component_t_15_17 <- ifelse((df_final$t_15_17==1),1,0) 
tmp$full_component_inv_16 <- ifelse((df_final$inv_16==1),1,0) 
tmp$full_component_t_8_21 <- ifelse((df_final$t_8_21==1),1,0) 
tmp$full_component_t_11 <- ifelse((df_final$t_v_11==1 | df_final$t_9_11==1),1,0)   
tmp$full_component_t_6_9 <- ifelse((df_final$t_6_9==1),1,0) 
tmp$full_component_inv_3 <- ifelse((df_final$inv_3==1),1,0)
tmp$full_component_additions <- ifelse((df_final$add_8==1 | df_final$add_13==1 | df_final$add_21==1 | df_final$add_22==1),1,0)
tmp$full_component_TP53_complex <- ifelse((df_final$TP53==1 | df_final$complex==1) ,1,0)
tmp$full_component_NPM1 <- ifelse((df_final$NPM1==1),1,0) 
tmp$full_component_CEBPA_bi <- ifelse((df_final$CEBPA_bi==1),1,0) 
tmp$full_component_DNMT3A_IDH1_2 <- ifelse((df_final$DNMT3A==1 & (df_final$IDH1==1 | df_final$IDH2_p.R140==1 | df_final$IDH2_p.R172==1)),1,0)
tmp$full_component_WT1 <- ifelse((df_final$WT1==1),1,0) 
tmp$full_component_chr_splicing_multiple <- ifelse((rowSums(df_final[,factors])>1) ,1,0)
tmp$full_component_chr_splicing_1 <- ifelse((rowSums(df_final[,factors])==1),1,0)
tmp$full_component_not_assigned <- ifelse(df_final$final_component=="not_assigned",1,0)
tmp$full_component_no_events <- ifelse(df_final$final_component=="no_events",1,0)

df_initial <- read.table("../../../data/initial_dataset/Master_04_10_2019.csv",sep = ',' , header = T)
rownames(df_initial) <- df_initial$data_pd
df_initial <- df_initial[,-1:-4]
df_eln <- read.table("../../../data/updated_dataset/eln_final.tsv",sep = '\t' , header = T)
rownames(df_eln) <- df_eln$X
df_eln <- df_eln[-1]
df <- merge(df_eln,tmp,by=0)
rownames(df) <- df$Row.names
df <- df[-1]
df <- merge(df,df_initial[,correlates],by=0)
rownames(df) <- df$Row.names
df <- df[-1]
for (col in colnames(df)[colSums(is.na(df)) > 0]){
    if(!is.element(col,c("os","os_status"))){        
        if(is.element(col,continuous_correlates)){
            df[,col][is.na(df[,col])] <- mean(df[,col], na.rm=TRUE)
        }else{
            df[,col][is.na(df[,col])] <- median(df[,col], na.rm=TRUE)
            }
    }
   
}
df <- na.omit(df)
df <- df[df$os>0,]

# Handling additions
df[df$full_component_additions==1 & (df$full_component_t_15_17==1 | df$full_component_inv_16==1 | df$full_component_t_8_21==1 | df$full_component_t_11==1 | df$full_component_t_6_9==1 | df$full_component_inv_3==1 | df$full_component_NPM1==1 | df$full_component_CEBPA_bi==1 | df$full_component_DNMT3A_IDH1_2==1 | df$full_component_WT1==1 ),"full_component_additions"] <- 0
df[df$full_component_additions==1 & (df$full_component_TP53_complex==1) & (df$del_7==1 | df$del_17==1 | df$del_5==1 | df$del_9==1 |df$del_13==1 | df$del_20==1 | df$del_18==1 | df$del_16==1 | df$del_12==1 | df$del_3==1),"full_component_additions"] <- 1

# Handling DNMT3A IDH
df[df$full_component_DNMT3A_IDH1_2==1 & (df$full_component_t_15_17==1 | df$full_component_inv_16==1 | df$full_component_t_8_21==1 | df$full_component_t_11==1 | df$full_component_t_6_9==1 | df$full_component_inv_3==1 | df$full_component_TP53_complex==1 | df$full_component_NPM1==1 | df$full_component_CEBPA_bi==1 | df$full_component_additions==1 ),"full_component_DNMT3A_IDH1_2"] <- 0

# Handling WT1
df[df$full_component_WT1==1 & (df$full_component_t_15_17==1 | df$full_component_inv_16==1 | df$full_component_t_8_21==1 | df$full_component_t_11==1 | df$full_component_t_6_9==1 | df$full_component_inv_3==1 | df$full_component_TP53_complex==1 | df$full_component_NPM1==1 | df$full_component_CEBPA_bi==1 ),"full_component_WT1"] <- 0

# Handling chr_splicing_1
df[df$full_component_chr_splicing_1==1 & (df$full_component_t_15_17==1 | df$full_component_inv_16==1 | df$full_component_t_8_21==1 | df$full_component_t_11==1 | df$full_component_t_6_9==1 | df$full_component_inv_3==1 | df$full_component_TP53_complex==1 | df$full_component_NPM1==1 | df$full_component_CEBPA_bi==1 ),"full_component_chr_splicing_1"] <- 0

df$overlap <- 0
df[rowSums(df[,colnames(df[,startsWith(colnames(df),
                                       
                                       "full_component_"),])])>1,"overlap"] <- 1
df <- df[,c(1:158,177:185,159:176,188,186:187)]
df$eln_2017 <- ifelse(df$eln_2017=="adverse",1,
                    ifelse(df$eln_2017=="intermediate",2,3))
write.table(df,"df_final_full_component.tsv")

In [149]:
colnames(df)

[1] "eln_2017"                            
  [2] "eln_2017_adverse"                    
  [3] "eln_2017_favorable"                  
  [4] "eln_2017_intermediate"               
  [5] "ASXL1"                               
  [6] "ASXL2"                               
  [7] "ASXL3"                               
  [8] "ATRX"                                
  [9] "BAGE3"                               
 [10] "BCOR"                                
 [11] "BRAF"                                
 [12] "CBFB"                                
 [13] "CBL"                                 
 [14] "CDKN2A"                              
 [15] "CEBPA_bi"                            
 [16] "CEBPA_mono"                          
 [17] "CNTN5"                               
 [18] "CREBBP"                              
 [19] "CSF1R"                               
 [20] "CSF3R"                               
 [21] "CTCF"                                
 [22] "CUL2"                                
 [23] "CUX1"                                
 [24] "DNMT3A"                              
 [25] "EED"                                 
 [26] "ETV6"                                
 [27] "EZH2"                                
 [28] "FBXW7"                               
 [29] "ITD"                                 
 [30] "FLT3_TKD"                            
 [31] "FLT3_other"                          
 [32] "GATA1"                               
 [33] "GATA2"                               
 [34] "GNAS"                                
 [35] "GNB1"                                
 [36] "IDH1"                                
 [37] "IDH2_p.R140"                         
 [38] "IDH2_p.R172"                         
 [39] "JAK2"                                
 [40] "JAK3"                                
 [41] "KANSL1"                              
 [42] "KDM6A"                               
 [43] "KIT"                                 
 [44] "KMT2C"                               
 [45] "KMT2D"                               
 [46] "KMT2E"                               
 [47] "KRAS"                                
 [48] "LUC7L2"                              
 [49] "MED12"                               
 [50] "MLL"                                 
 [51] "MPL"                                 
 [52] "MYC"                                 
 [53] "NF1"                                 
 [54] "NFE2"                                
 [55] "NOTCH1"                              
 [56] "NPM1"                                
 [57] "NRAS_other"                          
 [58] "NRAS_p.G12_13"                       
 [59] "NRAS_p.Q61_62"                       
 [60] "PDS5B"                               
 [61] "PHF6"                                
 [62] "PPFIA2"                              
 [63] "PRPF8"                               
 [64] "PTEN"                                
 [65] "PTPN11"                              
 [66] "PTPRF"                               
 [67] "PTPRT"                               
 [68] "RAD21"                               
 [69] "RIT1"                                
 [70] "RUNX1"                               
 [71] "S100B"                               
 [72] "SETBP1"                              
 [73] "SF1"                                 
 [74] "SF3B1"                               
 [75] "SMC1A"                               
 [76] "SMC3"                                
 [77] "SMG1"                                
 [78] "SPP1"                                
 [79] "SRSF2"                               
 [80] "STAG2"                               
 [81] "STAT5B"                              
 [82] "SUZ12"                               
 [83] "TET2"                                
 [84] "TP53"                                
 [85] "U2AF1_p.S34"                         
 [86] "U2AF1_p.Q157"                        
 [87] "WT1"                                 
 [88] "ZRSR2"                               
 [89] "add_8"                             